In [283]:
import urllib.request
from datetime import datetime, timedelta

In [284]:
url = "file:///Users/ph290/Downloads/Services%20-%20PharmOutcomes120121.html"
d = datetime.today()
output_filename = '/Users/ph290/Documents/HalloranSync/documents/for_mark/output_'+d.strftime('%Y-%m-%d')+'.csv'
x = urllib.request.urlopen(url)
data = str((x.read()))

In [288]:
out_dict = {}

d = datetime.today()

not_first_date = True

while not_first_date:
    date_given = d.strftime('%Y-%m-%d')
    if date_given in data:
        not_first_date = False
    else:
        d = d - timedelta(days=1)
    

date_exists = True

while date_exists:

    date_given = d.strftime('%Y-%m-%d')

    if date_given in data:

        not_end = True
        i=1

        while not_end:
            try:
                tmp = data.split(date_given)[i].split('NotClaimable')[0]
                try:
                    patient_name = tmp.split('info no-lock" title="')[1].split('">')[0]
                except IndexError:
                    patient_name = tmp.split('<span style="color:salmon;">&bull;&nbsp;')[1].split('<br>')[0]
                out_dict[patient_name]={}
                out_dict[patient_name]['Date Given'] = date_given
                out_dict[patient_name]['DOB'] = tmp.split('<br>&bull;&nbsp;')[1]
                out_dict[patient_name]['Postcode'] = tmp.split('<br>&bull;&nbsp;')[2].split('</span>')[0]
                out_dict[patient_name]['Vaccinator'] = tmp.split('provisionedit">')[5].split('</a>')[0]
                i += 1
            except IndexError:
                not_end = False

        d  =  d - timedelta(days=1)
    else:
        date_exists = False

In [290]:
#output the data
with open(output_filename, 'w') as f:
    f.write("%s,%s,%s,%s,%s,\n"%('Date Given','Patient name','DOB','Postcode','Vaccinator'))
    for key in out_dict.keys():
#         f.write("%s,%s\n"%(key,[out_dict[key][key2] for key2 in (out_dict[key])]))
        f.write("%s,%s,%s,%s,%s,\n"%(out_dict[key]['Date Given'],key,out_dict[key]['DOB'],out_dict[key]['Postcode'],out_dict[key]['Vaccinator']))
